<a href="https://colab.research.google.com/github/simasaadi/canada-climate-80yrs/blob/main/02_data_cleaning_reshaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd

# Always anchor to /content in Colab
os.chdir("/content")

REPO_DIR = Path("/content/canada-climate-80yrs")
RAW_PATH = REPO_DIR / "data" / "raw" / "canadian_climate_daily.csv"
PROCESSED_DIR = REPO_DIR / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("REPO_DIR exists:", REPO_DIR.exists(), "|", REPO_DIR)
print("RAW_PATH exists:", RAW_PATH.exists(), "| size:", (RAW_PATH.stat().st_size if RAW_PATH.exists() else None))
print("PROCESSED_DIR:", PROCESSED_DIR)


REPO_DIR exists: True | /content/canada-climate-80yrs
RAW_PATH exists: True | size: 3865257
PROCESSED_DIR: /content/canada-climate-80yrs/data/processed


In [20]:
df_raw = pd.read_csv(RAW_PATH)

print("Raw shape:", df_raw.shape)
print("Columns:", len(df_raw.columns))
display(df_raw.head(3))


Raw shape: (29221, 27)
Columns: 27


,LOCAL_DATE,MEAN_TEMPERATURE_CALGARY,TOTAL_PRECIPITATION_CALGARY,MEAN_TEMPERATURE_EDMONTON,TOTAL_PRECIPITATION_EDMONTON,MEAN_TEMPERATURE_HALIFAX,TOTAL_PRECIPITATION_HALIFAX,MEAN_TEMPERATURE_MONCTON,TOTAL_PRECIPITATION_MONCTON,MEAN_TEMPERATURE_MONTREAL,...,MEAN_TEMPERATURE_STJOHNS,TOTAL_PRECIPITATION_STJOHNS,MEAN_TEMPERATURE_TORONTO,TOTAL_PRECIPITATION_TORONTO,MEAN_TEMPERATURE_VANCOUVER,TOTAL_PRECIPITATION_VANCOUVER,MEAN_TEMPERATURE_WHITEHORSE,TOTAL_PRECIPITATION_WHITEHORSE,MEAN_TEMPERATURE_WINNIPEG,TOTAL_PRECIPITATION_WINNIPEG
0,01-Jan-1940 00:00:00,-11.4,0.5,NaN,NaN,NaN,NaN,-8.9,0.0,NaN,...,NaN,NaN,-8.9,0.0,8.9,5.8,NaN,NaN,-20.9,0.0
1,02-Jan-1940 00:00:00,-12.0,0.5,NaN,NaN,NaN,NaN,-14.5,0.0,NaN,...,NaN,NaN,-13.1,0.3,9.7,7.1,NaN,NaN,-18.4,0.0
2,03-Jan-1940 00:00:00,-12.0,1.0,NaN,NaN,NaN,NaN,-11.1,0.0,NaN,...,NaN,NaN,-6.1,0.0,7.8,1.0,NaN,NaN,-22.0,0.0


In [21]:
df = df_raw.copy()

if "LOCAL_DATE" not in df.columns:
    raise ValueError("Expected column LOCAL_DATE not found. Check your raw CSV.")

df["LOCAL_DATE"] = pd.to_datetime(df["LOCAL_DATE"], errors="coerce")

bad_dates = int(df["LOCAL_DATE"].isna().sum())
if bad_dates > 0:
    # Keep the rows but flag it (you can choose to drop later)
    print(f"Warning: {bad_dates} rows have unparsable LOCAL_DATE")

df = df.sort_values("LOCAL_DATE").reset_index(drop=True)

print("Date range:", df["LOCAL_DATE"].min(), "→", df["LOCAL_DATE"].max())
print("Duplicate LOCAL_DATE:", int(df["LOCAL_DATE"].duplicated().sum()))


Date range: 1940-01-01 00:00:00 → 2020-01-01 00:00:00
Duplicate LOCAL_DATE: 0


In [22]:
pattern = re.compile(r"^(MEAN_TEMPERATURE|TOTAL_PRECIPITATION)_(.+)$")

meta = []
for c in df.columns:
    if c == "LOCAL_DATE":
        continue
    m = pattern.match(c)
    if m:
        meta.append({"column": c, "variable": m.group(1), "city": m.group(2)})
    else:
        meta.append({"column": c, "variable": "OTHER", "city": None})

meta_df = pd.DataFrame(meta)

display(meta_df["variable"].value_counts().to_frame("n_columns"))
display(meta_df.query("variable != 'OTHER'").head(10))


,n_columns
variable,
MEAN_TEMPERATURE,13
TOTAL_PRECIPITATION,13


,column,variable,city
0,MEAN_TEMPERATURE_CALGARY,MEAN_TEMPERATURE,CALGARY
1,TOTAL_PRECIPITATION_CALGARY,TOTAL_PRECIPITATION,CALGARY
2,MEAN_TEMPERATURE_EDMONTON,MEAN_TEMPERATURE,EDMONTON
3,TOTAL_PRECIPITATION_EDMONTON,TOTAL_PRECIPITATION,EDMONTON
4,MEAN_TEMPERATURE_HALIFAX,MEAN_TEMPERATURE,HALIFAX
5,TOTAL_PRECIPITATION_HALIFAX,TOTAL_PRECIPITATION,HALIFAX
6,MEAN_TEMPERATURE_MONCTON,MEAN_TEMPERATURE,MONCTON
7,TOTAL_PRECIPITATION_MONCTON,TOTAL_PRECIPITATION,MONCTON
8,MEAN_TEMPERATURE_MONTREAL,MEAN_TEMPERATURE,MONTREAL
9,TOTAL_PRECIPITATION_MONTREAL,TOTAL_PRECIPITATION,MONTREAL


In [23]:
value_cols = meta_df.query("variable != 'OTHER'")["column"].tolist()

for c in value_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

print("Converted to numeric:", len(value_cols), "columns")


Converted to numeric: 26 columns


In [24]:
# Long format
df_long = (
    df.melt(id_vars=["LOCAL_DATE"], value_vars=value_cols,
            var_name="series", value_name="value")
)

# Parse series into variable + city
parsed = df_long["series"].str.extract(r"^(MEAN_TEMPERATURE|TOTAL_PRECIPITATION)_(.+)$")
df_long["variable"] = parsed[0]
df_long["city"] = parsed[1]

df_long = df_long.drop(columns=["series"])
df_long = df_long.rename(columns={"LOCAL_DATE": "local_date"})

# Pivot variables into separate columns
df_tidy = (
    df_long.pivot_table(
        index=["local_date", "city"],
        columns="variable",
        values="value",
        aggfunc="mean"
    )
    .reset_index()
)

# Clean column names
df_tidy.columns.name = None
df_tidy = df_tidy.rename(columns={
    "MEAN_TEMPERATURE": "mean_temperature",
    "TOTAL_PRECIPITATION": "total_precipitation",
})

print("Tidy shape:", df_tidy.shape)
display(df_tidy.head(5))


Tidy shape: (355513, 4)


,local_date,city,mean_temperature,total_precipitation
0,1940-01-01,CALGARY,-11.4,0.5
1,1940-01-01,MONCTON,-8.9,0.0
2,1940-01-01,OTTAWA,-17.0,0.0
3,1940-01-01,SASKATOON,-25.6,0.0
4,1940-01-01,TORONTO,-8.9,0.0


In [25]:
# How many cities?
n_cities = df_tidy["city"].nunique()
print("Cities:", n_cities)

# Missingness by variable
missing = df_tidy[["mean_temperature", "total_precipitation"]].isna().mean().mul(100).round(2)
print("Missing %:")
display(missing.to_frame("missing_pct"))

# Coverage per city
coverage_city = (
    df_tidy.groupby("city", as_index=False)
    .agg(
        n_days=("local_date", "nunique"),
        temp_nonnull=("mean_temperature", lambda s: int(s.notna().sum())),
        precip_nonnull=("total_precipitation", lambda s: int(s.notna().sum())),
    )
    .sort_values("n_days", ascending=False)
)

display(coverage_city.head(15))


Cities: 13
Missing %:


,missing_pct
mean_temperature,0.07
total_precipitation,1.28


,city,n_days,temp_nonnull,precip_nonnull
10,VANCOUVER,29179,29168,29166
9,TORONTO,29155,29147,29141
5,OTTAWA,29150,29145,29140
12,WINNIPEG,29104,29097,28974
0,CALGARY,29064,29032,29048
4,MONTREAL,28514,28466,28491
8,STJOHNS,28436,28413,28399
6,QUEBEC,28020,28007,27994
11,WHITEHORSE,27559,27530,25126
3,MONCTON,26885,26885,26454


In [26]:
df_tidy["year"] = df_tidy["local_date"].dt.year
df_tidy["month"] = df_tidy["local_date"].dt.month
df_tidy["dayofyear"] = df_tidy["local_date"].dt.dayofyear

# Example derived fields
df_tidy["is_freezing"] = df_tidy["mean_temperature"].lt(0)  # True/False where temp exists
df_tidy["precip_positive"] = df_tidy["total_precipitation"].gt(0)

display(df_tidy.head(5))


,local_date,city,mean_temperature,total_precipitation,year,month,dayofyear,is_freezing,precip_positive
0,1940-01-01,CALGARY,-11.4,0.5,1940,1,1,True,True
1,1940-01-01,MONCTON,-8.9,0.0,1940,1,1,True,False
2,1940-01-01,OTTAWA,-17.0,0.0,1940,1,1,True,False
3,1940-01-01,SASKATOON,-25.6,0.0,1940,1,1,True,False
4,1940-01-01,TORONTO,-8.9,0.0,1940,1,1,True,False


In [27]:
monthly = (
    df_tidy.groupby(["city", "year", "month"], as_index=False)
    .agg(
        mean_temp=("mean_temperature", "mean"),
        total_precip=("total_precipitation", "sum"),
        n_obs_temp=("mean_temperature", lambda s: int(s.notna().sum())),
        n_obs_precip=("total_precipitation", lambda s: int(s.notna().sum())),
    )
)

yearly = (
    df_tidy.groupby(["city", "year"], as_index=False)
    .agg(
        mean_temp=("mean_temperature", "mean"),
        total_precip=("total_precipitation", "sum"),
        n_obs_temp=("mean_temperature", lambda s: int(s.notna().sum())),
        n_obs_precip=("total_precipitation", lambda s: int(s.notna().sum())),
    )
)

print("Monthly shape:", monthly.shape)
print("Yearly shape:", yearly.shape)
display(monthly.head(5))
display(yearly.head(5))


Monthly shape: (11731, 7)
Yearly shape: (998, 6)


,city,year,month,mean_temp,total_precip,n_obs_temp,n_obs_precip
0,CALGARY,1940,1,-11.912903,6.2,31,31
1,CALGARY,1940,2,-10.820690,22.6,29,29
2,CALGARY,1940,3,-2.816129,21.0,31,31
3,CALGARY,1940,4,1.370000,89.9,30,30
4,CALGARY,1940,5,11.729032,27.5,31,31


,city,year,mean_temp,total_precip,n_obs_temp,n_obs_precip
0,CALGARY,1940,3.704098,456.7,366,366
1,CALGARY,1941,4.907671,392.1,365,365
2,CALGARY,1942,3.994795,522.5,365,365
3,CALGARY,1943,4.458082,336.3,365,365
4,CALGARY,1944,4.964208,432.9,366,366


In [28]:
out_tidy = PROCESSED_DIR / "climate_daily_tidy.parquet"
out_monthly = PROCESSED_DIR / "climate_monthly.parquet"
out_yearly = PROCESSED_DIR / "climate_yearly.parquet"
out_meta = PROCESSED_DIR / "series_metadata.csv"

df_tidy.to_parquet(out_tidy, index=False)
monthly.to_parquet(out_monthly, index=False)
yearly.to_parquet(out_yearly, index=False)
meta_df.to_csv(out_meta, index=False)

print("Saved:")
for p in [out_tidy, out_monthly, out_yearly, out_meta]:
    print(" -", p)


Saved:
 - /content/canada-climate-80yrs/data/processed/climate_daily_tidy.parquet
 - /content/canada-climate-80yrs/data/processed/climate_monthly.parquet
 - /content/canada-climate-80yrs/data/processed/climate_yearly.parquet
 - /content/canada-climate-80yrs/data/processed/series_metadata.csv


In [29]:
check = pd.read_parquet(out_tidy)
print("Reloaded tidy:", check.shape)
display(check.head(3))


Reloaded tidy: (355513, 9)


,local_date,city,mean_temperature,total_precipitation,year,month,dayofyear,is_freezing,precip_positive
0,1940-01-01,CALGARY,-11.4,0.5,1940,1,1,True,True
1,1940-01-01,MONCTON,-8.9,0.0,1940,1,1,True,False
2,1940-01-01,OTTAWA,-17.0,0.0,1940,1,1,True,False
